In [1]:
import sys, os, time
import sqlite3
import pickle
import numpy as np


In [2]:
root = '/home/fleeb/workspace/ml_datasets/wiki/'

In [3]:
#conn = sqlite3.connect(os.path.join(root, 'wiki_links.db'))
conn = sqlite3.connect('example.db')

In [4]:
c = conn.cursor()

In [41]:
c.execute('''CREATE TABLE articles (
name TEXT PRIMARY KEY,
num INT)''')

In [5]:
c.execute('''PRAGMA table_info([articles]);''')
print(c.fetchall())

[(0, 'name', 'TEXT', 0, None, 1), (1, 'num', 'INT', 0, None, 0)]


In [6]:
ts = [['k1'], ['k3'], ['k4']]

In [9]:
c.executemany(_insert_temp, ts)

In [10]:
c.execute('''SELECT * FROM articles;''')
print(c.fetchall())

[('t1', 6), ('t2', 4), ('t3', 1), ('k1', 1), ('k3', 1), ('k4', 1)]


In [38]:
conn.commit()

In [ ]:
conn.close()

In [17]:

_inc_temp = '''UPDATE articles SET num = num + 1 WHERE name IN ({})'''
_insert_temp = '''INSERT INTO articles VALUES (?,1);'''
_sel_id_temp = '''SELECT rowid FROM articles WHERE name IN ({})'''
_sel_name_temp = '''SELECT name FROM articles WHERE rowid IN ({})'''
_sel_full_id_temp = '''SELECT * FROM articles WHERE rowid IN ({})'''
_sel_full_name_temp = '''SELECT * FROM articles WHERE name IN ({})'''

class Articles(object):
    def __init__(self, db_path, lock=None):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.lock = None
        
    def names(self):
        self.cursor.execute('''SELECT name FROM articles''')
        return iter(self.cursor)
    
    def nums(self):
        self.cursor.execute('''SELECT name FROM articles''')
        return iter(self.cursor)
    
    def items(self):
        self.cursor.execute('''SELECT * FROM articles''')
        return iter(self.cursor)
        
    def add(self, *names):
        if self.lock is not None:
            self.lock.acquire()
        for name in names:
            try:
                #print(_insert_temp, name)
                self.cursor.execute(_insert_temp, (name,))
            except sqlite3.IntegrityError:
                pass
        #self.cursor.executemany(_insert_temp, [(n,) for n in names])
        self.conn.commit()
        if self.lock is not None:
            self.lock.release()
        
    def increment(self, *names):
        if self.lock is not None:
            self.lock.acquire()
        self.cursor.execute(_inc_temp.format(', '.join('?'*len(names))), names)
        self.conn.commit()
        if self.lock is not None:
            self.lock.release()
        
    def __len__(self):
        self.cursor.execute('''SELECT COUNT(*) FROM articles''')
        return self.cursor.fetchall()[0][0]
        
    def get_id(self, *names):
        self.cursor.execute(_sel_id_temp.format(', '.join('?'*len(names))), names)
        return self.cursor.fetchall()
        
    def get_name(self, *IDs):
        self.cursor.execute(_sel_name_temp.format(', '.join('?'*len(IDs))), IDs)
        return self.cursor.fetchall()
    
    def get_full_from_names(self, *names):
        self.cursor.execute(_sel_full_name_temp.format(', '.join('?'*len(names))), names)
        return self.cursor.fetchall()
    
    def get_full_from_ids(self, *IDs):
        self.cursor.execute(_sel_full_id_temp.format(', '.join('?'*len(IDs))), IDs)
        return self.cursor.fetchall()
    
    def __del__(self):
        self.conn.close()

In [18]:
db = Articles('example.db')

In [19]:
list(db.items())

[('t1', 6), ('t2', 4), ('t3', 1)]

In [20]:
len(db)

3

In [21]:
db.add('t1', 't3')

In [22]:
len(db)

3

In [23]:
db.increment('t2')

In [24]:
list(db.items())

[('t1', 6), ('t2', 5), ('t3', 1)]